<a href="https://colab.research.google.com/github/CH2-PS181/ScanProspecta/blob/main/Machine%20Learning/Resume_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ScanProspecta**

### **Import Library**

In [1]:
!pip install wget
import wget
!pip install datasets
from datasets import load_dataset
import pandas as pd
import pandas as pd
import wget
wget.download("https://raw.githubusercontent.com/yogawicaksana/helper_prabowo/main/helper_prabowo_ml.py", out="helper_prabowo_ml.py")
from helper_prabowo_ml import clean_html, remove_links, remove_special_characters, removeStopWords, remove_, remove_digits, lower, email_address, non_ascii, punct
from sklearn.model_selection import train_test_split
!pip install transformers
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### **Download Dataset**

In [2]:
!wget https://huggingface.co/datasets/InferencePrince555/Resume-Dataset/resolve/main/updated_data_final_cleaned.csv

--2023-12-07 07:00:18--  https://huggingface.co/datasets/InferencePrince555/Resume-Dataset/resolve/main/updated_data_final_cleaned.csv
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.37, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8a/ae/8aae04b8e17069856ddb28e3052e381c44ae2dc21356336bff549473bfda5ad1/c821343fe6a2a0e1bb70a08122615936c16af045981d632e6708a72977186c19?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27updated_data_final_cleaned.csv%3B+filename%3D%22updated_data_final_cleaned.csv%22%3B&response-content-type=text%2Fcsv&Expires=1702191618&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjE5MTYxOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84YS9hZS84YWFlMDRiOGUxNzA2OTg1NmRkYjI4ZTMwNTJlMzgxYzQ0YWUyZGMyMTM1NjMzNmJmZjU0OTQ3M2JmZGE1

In [3]:
file_path = '/content/updated_data_final_cleaned.csv'
preview = pd.read_csv(file_path)
print(preview.head())

                              instruction  input  \
0  Generate a Resume for a Accountant Job    NaN   
1  Generate a Resume for a Accountant Job    NaN   
2  Generate a Resume for a Accountant Job    NaN   
3  Generate a Resume for a Accountant Job    NaN   
4  Generate a Resume for a Accountant Job    NaN   

                                         Resume_test  
0  ACCOUNTANT Professional Summary Results orient...  
1  STAFF ACCOUNTANT Summary Flexible Accountant w...  
2  STAFF ACCOUNTANT Summary Highly analytical and...  
3  SENIOR ACCOUNTANT Summary A highly competent m...  
4  SENIOR ACCOUNTANT Summary 11 years experience ...  


### **Load Dataset**

In [4]:
# Load the CSV file into a Pandas DataFrame
data = pd.read_csv(file_path)

# Function to extract job positions from 'instruction' column
def extract_job_position(instruction):
    # Split the text between 'for a' and 'Job'
    start = 'for a '
    end = ' Job'
    job_position = instruction.split(start)[1].split(end)[0]
    return job_position.lower()  # Convert to lowercase

# Apply function to extract job positions and update 'instruction' column
data['instruction'] = data['instruction'].apply(extract_job_position)

### **Preview Raw Data**

In [5]:
# Display the updated DataFrame
print(data.tail())

         instruction  input                                        Resume_test
32476  web developer    NaN  Software Engineer span lSoftwarespan Engineer ...
32477  web developer    NaN  Sr Systems Manager Sr Business Manager Sr span...
32478  web developer    NaN  Full Stack NET Developer Full Stack NET span l...
32479  web developer    NaN  Director of Information Systems Director of In...
32480  web developer    NaN  UX Engineer UX Engineer UX Engineer Redmond WA...


### **Preprocess Data**

In [6]:
# Function to preprocess text using helper_prabowo_ml functions
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Apply text preprocessing functions
        text = clean_html(text)
        text = remove_links(text)
        text = remove_special_characters(text)
        text = removeStopWords(text)
        text = remove_(text)
        text = remove_digits(text)
        text = lower(text)
        text = email_address(text)
        text = non_ascii(text)
        text = punct(text)
        return text
    else:
        return text

# Apply function to preprocess 'Resume_test' column
data['Resume_test'] = data['Resume_test'].apply(preprocess_text)

### **Preview Processed Data**

In [7]:
# Display the updated DataFrame
print(data.tail())

         instruction  input                                        Resume_test
32476  web developer    NaN  software engineer span lsoftwarespan engineer ...
32477  web developer    NaN  sr systems manager sr business manager sr span...
32478  web developer    NaN  full stack net developer full stack net span l...
32479  web developer    NaN  director information systems director informat...
32480  web developer    NaN  ux engineer ux engineer ux engineer redmond wa...


### **Split Data**

In [8]:
# Splitting the data into training (80%) and temporary (20%) sets
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# Further splitting the temporary set into validation (50%) and test (50%) sets
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Displaying the shapes of the resulting sets
print("Train set shape:", train_data.shape)
print("Validation set shape:", val_data.shape)
print("Test set shape:", test_data.shape)

Train set shape: (25984, 3)
Validation set shape: (3248, 3)
Test set shape: (3249, 3)


### **Tokenize**

In [9]:
# Set the desired maximum length to 6400
max_len = 6400

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data and pad sequences
def tokenize_and_pad(text):
    if isinstance(text, str):
        encoded_dict = tokenizer.encode_plus(
            text[:max_len],
            max_length=max_len,
            truncation=True,
            padding='max_length',
            return_tensors='tf'
        )
        return encoded_dict['input_ids'], encoded_dict['attention_mask']
    else:
        return tf.constant([], dtype=tf.int32), tf.constant([], dtype=tf.int32)

# Filter data yang tidak kosong pada kolom 'Resume_test'
train_data = train_data[train_data['Resume_test'].notnull()]
val_data = val_data[val_data['Resume_test'].notnull()]
test_data = test_data[test_data['Resume_test'].notnull()]

# Apply tokenization and padding to the dataset
train_data['input_ids'], train_data['attention_mask'] = zip(*train_data['Resume_test'].apply(tokenize_and_pad))
val_data['input_ids'], val_data['attention_mask'] = zip(*val_data['Resume_test'].apply(tokenize_and_pad))
test_data['input_ids'], test_data['attention_mask'] = zip(*test_data['Resume_test'].apply(tokenize_and_pad))

### **Build Model**

In [10]:
# Take the number of classes or labels from the 'instruction' column
num_classes = len(train_data['instruction'].unique())

# Define the BERT model architecture
def build_model():
    input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    bert_outputs = bert_model([input_ids, attention_mask])[1]  # Getting pooled outputs
    dropout = tf.keras.layers.Dropout(0.1)(bert_outputs)
    dense = tf.keras.layers.Dense(64, activation='relu')(dropout)
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(dense)
    model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)
    return model

# Initialize and compile the model
model = build_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

### **Train Model**

In [ ]:
# Train the model
model.fit(
    [pad_sequences(np.vstack(train_data['input_ids']), maxlen=max_len, dtype='long', truncating='post', padding='post'),
     pad_sequences(np.vstack(train_data['attention_mask']), maxlen=max_len, dtype='long', truncating='post', padding='post')],
    train_data['instruction'],
    validation_data=(
        [pad_sequences(np.vstack(val_data['input_ids']), maxlen=max_len, dtype='long', truncating='post', padding='post'),
         pad_sequences(np.vstack(val_data['attention_mask']), maxlen=max_len, dtype='long', truncating='post', padding='post')],
        val_data['instruction']
    ),
    epochs=3,
    batch_size=32
)